<a href="https://colab.research.google.com/github/bluebluer/hello-wolrd/blob/master/%E7%BA%BF%E6%80%A7%E5%9B%9E%E5%BD%923_1%263_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install torch

In [4]:
import torch
from time import time

In [6]:
print(torch.__version__)

1.10.0+cu111


In [7]:
a = torch.ones(500)
b = torch.ones(500)

In [8]:
start = time()
c = torch.zeros(500)
for i in range(500):
  c[i] = a[i] + b[i]
print(time()-start)

0.04432821273803711


In [9]:
# 矢量加法
start = time()
d = a + b
print(time()-start)

0.00027108192443847656


In [10]:
# 广播机制
a = torch.ones(3)
b = 10
print(a+b)

tensor([11., 11., 11.])


In [11]:
% matplotlib inline
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

In [ ]:
print(torch.__version__)

In [14]:
# 生成数据集
num_inputs = 2
num_examples = 1000
true_w = [2,-3.4]
true_b = 4.2
features = torch.randn(num_examples,num_inputs,dtype=torch.float32)
labels = true_w[0] * features[:,0] + true_w[1] * features[:,1] +true_b
labels += torch.tensor(np.random.normal(0,0.01,size=labels.size()),dtype=torch.float32)

In [15]:
print(features[0],labels[0])


tensor([0.1385, 0.7224]) tensor(2.0151)


In [16]:
def use_svg_display():
  display.set_matplotlib_fromats('svg')


In [17]:
def set_figsize(figsize=(3.5,2.5)):
  use_svg_display()
  plt.rcParams['figure.figsize'] = figsize

In [18]:
# 本函数已保存在d2lzh包中方便以后使用
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)  # 样本的读取顺序是随机的
    for i in range(0, num_examples, batch_size):
        j = torch.LongTensor(indices[i: min(i + batch_size, num_examples)]) # 最后一次可能不足一个batch
        yield  features.index_select(0, j), labels.index_select(0, j)

In [19]:
batch_size = 10

for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break

tensor([[-1.4507, -0.0407],
        [ 0.3160,  1.1521],
        [-0.3605,  1.6033],
        [-0.7852,  0.0741],
        [-1.0097, -2.7993],
        [ 0.9157, -1.4110],
        [-1.8243,  0.8907],
        [-0.6469, -0.1665],
        [ 0.3551, -0.2741],
        [ 1.1066,  1.1821]]) 
 tensor([ 1.4562,  0.9248, -1.9709,  2.3881, 11.7059, 10.8299, -2.4678,  3.4698,
         5.8183,  2.3984])


In [20]:
# 初始化模型参数
w = torch.tensor(np.random.normal(0, 0.01, (num_inputs, 1)), dtype=torch.float32)
b = torch.zeros(1, dtype=torch.float32)

In [21]:
w.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True) 

tensor([0.], requires_grad=True)

In [22]:
# 定义模型
def linreg(X, w, b):  # 本函数已保存在d2lzh包中方便以后使用
    return torch.mm(X, w) + b

In [23]:
# 定义损失函数
def squared_loss(y_hat, y):  # 本函数已保存在pytorch_d2lzh包中方便以后使用
    return (y_hat - y.view(y_hat.size())) ** 2 / 2

In [24]:
# 定义优化算法
def sgd(params, lr, batch_size):  # 本函数已保存在d2lzh_pytorch包中方便以后使用
    for param in params:
        param.data -= lr * param.grad / batch_size # 注意这里更改param时用的param.data

In [25]:
# 训练模型
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

for epoch in range(num_epochs):  # 训练模型一共需要num_epochs个迭代周期
    # 在每一个迭代周期中，会使用训练数据集中所有样本一次（假设样本数能够被批量大小整除）。X
    # 和y分别是小批量样本的特征和标签
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y).sum()  # l是有关小批量X和y的损失
        l.backward()  # 小批量的损失对模型参数求梯度
        sgd([w, b], lr, batch_size)  # 使用小批量随机梯度下降迭代模型参数
        
        # 不要忘了梯度清零
        w.grad.data.zero_()
        b.grad.data.zero_()
    train_l = loss(net(features, w, b), labels)
    print('epoch %d, loss %f' % (epoch + 1, train_l.mean().item()))

epoch 1, loss 0.041779
epoch 2, loss 0.000171
epoch 3, loss 0.000055


In [26]:
print(true_w, '\n', w)
print(true_b, '\n', b)

[2, -3.4] 
 tensor([[ 2.0002],
        [-3.3996]], requires_grad=True)
4.2 
 tensor([4.1995], requires_grad=True)
